https://www.youtube.com/watch?v=jtIMnmbnOFo&t=103s

In [ ]:
# Install the transformers library
!pip install transformers
# instal social media scraper
!pip install snscrape

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.4 MB 29.9 MB/s 
     |████████████████████████████████| 6.6 MB 56.0 MB/s 
     |████████████████████████████████| 101 kB 13.5 MB/s 
     |████████████████████████████████| 596 kB 64.6 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# Import Dependencies


In [ ]:
import snscrape.modules.twitter as sntwitter
import pandas as pd
import datetime
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from scipy.special import softmax

# Initialize Sentiment analysis model

In [ ]:
# load model and tokenizer
roberta = "cardiffnlp/twitter-roberta-base-sentiment"

model = AutoModelForSequenceClassification.from_pretrained(roberta)
tokenizer = AutoTokenizer.from_pretrained(roberta)

Downloading:   0%|          | 0.00/747 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/476M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/150 [00:00<?, ?B/s]

# Scrape twitter Advanced search link
#### https://twitter.com/search-advanced

In [ ]:
query = "Tesla until:2021-03-10 since:2021-01-10"
tweets = []
limit = 2000


for tweet in sntwitter.TwitterSearchScraper(query).get_items():
    
    # print(vars(tweet))
    # break
    if len(tweets) == limit:
        break
    else:
        tweets.append([tweet.date, tweet.username, tweet.content])
        
df = pd.DataFrame(tweets, columns=['Date', 'User', 'Tweet'])
df

# NLP Sentiment

Input the text to return a sentiment value

In [ ]:
score_list = []
for idx, j in enumerate(df['Tweet']):
  tweet = j

  # precprcess tweet
  tweet_words = []

  for word in tweet.split(' '):
      if word.startswith('@') and len(word) > 1:
          word = '@user'
      
      elif word.startswith('http'):
          word = "http"
      tweet_words.append(word)

  tweet_proc = " ".join(tweet_words)


  # labels = ['Negative', 'Neutral', 'Positive']

  # sentiment analysis
  encoded_tweet = tokenizer(tweet_proc, return_tensors='pt')
  # output = model(encoded_tweet['input_ids'], encoded_tweet['attention_mask'])
  output = model(**encoded_tweet)

  scores = output[0][0].detach().numpy()
  scores = softmax(scores)

  score_list.append(scores)

In [ ]:
labels = ['Negative', 'Neutral', 'Positive']
sentiment = pd.DataFrame(score_list, columns = labels)

for i in labels:
  df[i] = sentiment[i]

df['Sentiment'] = df.iloc[:, 3:6].idxmax(axis=1)

In [ ]:
df1 = df.copy() 

In [ ]:
df1['D1'] = df1.Date.dt.date
df1['D2'] = df1.Date.dt.time

In [ ]:
df1


In [ ]:
# group the dates and sum the volume of sentiment values
valueC = df1.Sentiment.value_counts().to_frame()

import seaborn as sns
sns.set_theme(style="whitegrid")
ax = sns.barplot(x=data.index, y=data.count )
# line or bar plot the volume to find some activity 

In [ ]:
# get stock times for technical analisys
# observe for tweet volume behaviour 

Cant be right without a whole weeks or 5 day observations containing. speed issues might be google colab

# Scraping Link reference https://www.youtube.com/watch?v=DqtlR0y0suo&t=225s

Train a transformer NLP model 

In [ ]:
# Import required packages
import torch
import pandas as pd
import numpy as np
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer

# Create class for data preparation
class SimpleDataset:
    def __init__(self, tokenized_texts):
        self.tokenized_texts = tokenized_texts
    
    def __len__(self):
        return len(self.tokenized_texts["input_ids"])
    
    def __getitem__(self, idx):
        return {k: v[idx] for k, v in self.tokenized_texts.items()}

In [ ]:
# Load tokenizer and model, create trainer
model_name = "siebert/sentiment-roberta-large-english"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)
trainer = Trainer(model=model)

In [ ]:
# Create list of texts (can be imported from .csv, .xls etc.)
pred_texts = ['I like that','That is annoying','This is great!','Wouldn´t recommend it.']

In [ ]:
# Example: Import data from csv-file stored on Google Drive

#from google.colab import drive
#drive.mount('/content/drive')


#file_name = "/content/drive/MyDrive/Colab Notebooks/your-filename.csv"
#text_column = "text"

#df_pred = pd.read_csv(file_name)
#pred_texts = df_pred[text_column].dropna().astype('str').tolist()

In [ ]:
# Tokenize texts and create prediction data set
tokenized_texts = tokenizer(pred_texts,truncation=True,padding=True)
pred_dataset = SimpleDataset(tokenized_texts)

In [ ]:
# Run predictions
predictions = trainer.predict(pred_dataset)

In [ ]:
# Transform predictions to labels
preds = predictions.predictions.argmax(-1)
labels = pd.Series(preds).map(model.config.id2label)
scores = (np.exp(predictions[0])/np.exp(predictions[0]).sum(-1,keepdims=True)).max(1)

In [ ]:
# Create DataFrame with texts, predictions, labels, and scores
df = pd.DataFrame(list(zip(pred_texts,preds,labels,scores)), columns=['text','pred','label','score'])
df.head()